# PT100 Temperature Analysis

This notebook loads temperature data from TSV files and displays interactive graphs.

In [1]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from pathlib import Path

## Two-Point Calibration Analysis

Load the new measurement file and perform calibration analysis.

In [2]:
# Load the calibration measurement file
cal_tsv_path = Path("./calibration_data.tsv")
df_cal = pd.read_csv(cal_tsv_path, sep='\t', parse_dates=['timestamp'])

# Create relative time column (seconds from start)
t0 = df_cal['timestamp'].iloc[0]
df_cal['time_rel'] = (df_cal['timestamp'] - t0).dt.total_seconds()

print(f"Loaded {len(df_cal)} rows")
print(f"Time range: {df_cal['timestamp'].min()} to {df_cal['timestamp'].max()}")
print(f"Duration: {df_cal['timestamp'].max() - df_cal['timestamp'].min()}")
print(f"Start time (t=0): {t0}")
df_cal.head()

Loaded 15116 rows
Time range: 2025-12-17 15:22:55.402000+00:00 to 2025-12-17 16:32:11.497000+00:00
Duration: 0 days 01:09:16.095000
Start time (t=0): 2025-12-17 15:22:55.402000+00:00


,timestamp,boot_id,sampling_resolution_bits,sampling_sample_time_cycles,sampling_oversampling,sampling_n_measurements,sampling_amplification,pt100_1_series_resistor_ohms,pt100_2_series_resistor_ohms,raw_pt100_1_p20,raw_pt100_1_median,raw_pt100_1_p80,raw_pt100_2_p20,raw_pt100_2_median,raw_pt100_2_p80,pt100_1_r_pt,pt100_1_temperature,pt100_2_r_pt,pt100_2_temperature,time_rel
0,2025-12-17 15:22:55.402000+00:00,3143733478,12,640,256,30,16,3183.6,3203.3,31412,31418,31420,31470,31475,31480,98.33666,-4.320358,99.13023,-2.259133,0.000
1,2025-12-17 15:22:55.677000+00:00,3143733478,12,640,256,30,16,3183.6,3203.3,31410,31414,31416,31468,31472,31475,98.32375,-4.353888,99.12049,-2.284439,0.275
2,2025-12-17 15:22:55.952000+00:00,3143733478,12,640,256,30,16,3183.6,3203.3,31408,31412,31418,31467,31472,31477,98.31730,-4.370652,99.12049,-2.284439,0.550
3,2025-12-17 15:22:56.227000+00:00,3143733478,12,640,256,30,16,3183.6,3203.3,31410,31414,31419,31465,31471,31477,98.32375,-4.353888,99.11724,-2.292881,0.825
4,2025-12-17 15:22:56.502000+00:00,3143733478,12,640,256,30,16,3183.6,3203.3,31413,31416,31423,31474,31477,31480,98.33021,-4.337122,99.13673,-2.242269,1.100


### Raw ADC Values with Moving Average Filter

Plot raw median values for both PT100 sensors with a 5-second moving average overlay.

In [3]:
# Calculate approximate sample rate and window size for 5-second moving average
time_diffs = df_cal['timestamp'].diff().dt.total_seconds().dropna()
avg_sample_interval = time_diffs.median()
print(f"Average sample interval: {avg_sample_interval:.3f} seconds")

# Calculate window size for ~5 second moving average
window_size = max(1, int(5.0 / avg_sample_interval))
print(f"Window size for 5s moving average: {window_size} samples")

# Apply moving average filter
df_cal['pt100_1_median_filtered'] = df_cal['raw_pt100_1_median'].rolling(window=window_size, center=True).mean()
df_cal['pt100_2_median_filtered'] = df_cal['raw_pt100_2_median'].rolling(window=window_size, center=True).mean()

Average sample interval: 0.275 seconds
Window size for 5s moving average: 18 samples


In [4]:
# Create interactive plot with raw and filtered data
fig_raw = go.Figure()

# Raw PT100 #1 (light, semi-transparent)
fig_raw.add_trace(go.Scatter(
    x=df_cal['time_rel'],
    y=df_cal['raw_pt100_1_median'],
    mode='lines',
    name='PT100 #1 Raw',
    line=dict(color='rgba(31, 119, 180, 0.3)', width=1),
))

# Filtered PT100 #1
fig_raw.add_trace(go.Scatter(
    x=df_cal['time_rel'],
    y=df_cal['pt100_1_median_filtered'],
    mode='lines',
    name='PT100 #1 Filtered (5s)',
    line=dict(color='#1f77b4', width=2),
))

# Raw PT100 #2 (light, semi-transparent)
fig_raw.add_trace(go.Scatter(
    x=df_cal['time_rel'],
    y=df_cal['raw_pt100_2_median'],
    mode='lines',
    name='PT100 #2 Raw',
    line=dict(color='rgba(255, 127, 14, 0.3)', width=1),
))

# Filtered PT100 #2
fig_raw.add_trace(go.Scatter(
    x=df_cal['time_rel'],
    y=df_cal['pt100_2_median_filtered'],
    mode='lines',
    name='PT100 #2 Filtered (5s)',
    line=dict(color='#ff7f0e', width=2),
))

fig_raw.update_layout(
    title='Raw ADC Median Values (with 5s Moving Average)',
    xaxis_title='Time (seconds)',
    yaxis_title='Raw ADC Value',
    hovermode='x unified',
    legend=dict(yanchor="top", y=0.99, xanchor="left", x=0.01),
    dragmode='zoom',  # Enable box zoom by default
)

# Show with config enabling scroll zoom on both axes
fig_raw.show(config={'scrollZoom': True})

### Two-Point Calibration

Calibration points (relative to start):
- **Ice point (0°C)**: t = 100.00s
- **Boiling point (99.6°C)**: t = 3153.77s (adjusted for ~120m elevation in Frankfurt am Main)

Using 10-second averaging windows around each time point.

In [5]:
# Define calibration time points and known temperatures
import numpy as np

# Calibration parameters (relative time in seconds from start)
# Original timestamps: ice=13:26:24, boil=14:30:24, start=13:20:28.790
cal_ice_time_rel =100.00  # seconds
cal_boil_time_rel = 3153.77  # seconds
cal_ice_temp = 0.0  # °C
cal_boil_temp = 99.6  # °C (adjusted for ~120m elevation)

# Averaging window (±5 seconds = 10 second total window)
avg_window_seconds = 5

# Extract data within averaging windows
ice_mask = (df_cal['time_rel'] >= cal_ice_time_rel - avg_window_seconds) & \
           (df_cal['time_rel'] <= cal_ice_time_rel + avg_window_seconds)
boil_mask = (df_cal['time_rel'] >= cal_boil_time_rel - avg_window_seconds) & \
            (df_cal['time_rel'] <= cal_boil_time_rel + avg_window_seconds)

print(f"Ice point window: {ice_mask.sum()} samples")
print(f"Boiling point window: {boil_mask.sum()} samples")

# Calculate average raw ADC values at calibration points
pt100_1_ice_adc = df_cal.loc[ice_mask, 'raw_pt100_1_median'].mean()
pt100_1_boil_adc = df_cal.loc[boil_mask, 'raw_pt100_1_median'].mean()
pt100_2_ice_adc = df_cal.loc[ice_mask, 'raw_pt100_2_median'].mean()
pt100_2_boil_adc = df_cal.loc[boil_mask, 'raw_pt100_2_median'].mean()

print(f"\n=== PT100 #1 Calibration ADC Values ===")
print(f"Ice (0°C):      {pt100_1_ice_adc:.1f} ADC")
print(f"Boiling (99.6°C): {pt100_1_boil_adc:.1f} ADC")

print(f"\n=== PT100 #2 Calibration ADC Values ===")
print(f"Ice (0°C):      {pt100_2_ice_adc:.1f} ADC")
print(f"Boiling (99.6°C): {pt100_2_boil_adc:.1f} ADC")

# Calculate linear calibration coefficients: T = slope * ADC + offset
# slope = (T2 - T1) / (ADC2 - ADC1)
# offset = T1 - slope * ADC1

pt100_1_slope = (cal_boil_temp - cal_ice_temp) / (pt100_1_boil_adc - pt100_1_ice_adc)
pt100_1_offset = cal_ice_temp - pt100_1_slope * pt100_1_ice_adc

pt100_2_slope = (cal_boil_temp - cal_ice_temp) / (pt100_2_boil_adc - pt100_2_ice_adc)
pt100_2_offset = cal_ice_temp - pt100_2_slope * pt100_2_ice_adc

print(f"\n=== Calibration Coefficients ===")
print(f"PT100 #1: T = {pt100_1_slope:.6f} * ADC + ({pt100_1_offset:.3f})")
print(f"PT100 #2: T = {pt100_2_slope:.6f} * ADC + ({pt100_2_offset:.3f})")

# Apply calibration to get corrected temperatures
df_cal['pt100_1_temp_calibrated'] = pt100_1_slope * df_cal['raw_pt100_1_median'] + pt100_1_offset
df_cal['pt100_2_temp_calibrated'] = pt100_2_slope * df_cal['raw_pt100_2_median'] + pt100_2_offset

# Verify calibration at reference points
pt100_1_ice_check = df_cal.loc[ice_mask, 'pt100_1_temp_calibrated'].mean()
pt100_1_boil_check = df_cal.loc[boil_mask, 'pt100_1_temp_calibrated'].mean()
pt100_2_ice_check = df_cal.loc[ice_mask, 'pt100_2_temp_calibrated'].mean()
pt100_2_boil_check = df_cal.loc[boil_mask, 'pt100_2_temp_calibrated'].mean()

print(f"\n=== Calibration Verification ===")
print(f"PT100 #1 at ice point:     {pt100_1_ice_check:.3f}°C (expected: {cal_ice_temp}°C)")
print(f"PT100 #1 at boiling point: {pt100_1_boil_check:.3f}°C (expected: {cal_boil_temp}°C)")
print(f"PT100 #2 at ice point:     {pt100_2_ice_check:.3f}°C (expected: {cal_ice_temp}°C)")
print(f"PT100 #2 at boiling point: {pt100_2_boil_check:.3f}°C (expected: {cal_boil_temp}°C)")

Ice point window: 36 samples
Boiling point window: 37 samples

=== PT100 #1 Calibration ADC Values ===
Ice (0°C):      31392.4 ADC
Boiling (99.6°C): 42959.5 ADC

=== PT100 #2 Calibration ADC Values ===
Ice (0°C):      31463.8 ADC
Boiling (99.6°C): 42957.9 ADC

=== Calibration Coefficients ===
PT100 #1: T = 0.008611 * ADC + (-270.310)
PT100 #2: T = 0.008665 * ADC + (-272.644)

=== Calibration Verification ===
PT100 #1 at ice point:     -0.000°C (expected: 0.0°C)
PT100 #1 at boiling point: 99.600°C (expected: 99.6°C)
PT100 #2 at ice point:     0.000°C (expected: 0.0°C)
PT100 #2 at boiling point: 99.600°C (expected: 99.6°C)


In [6]:
# Plot calibrated temperatures
fig_cal = go.Figure()

# PT100 #1 calibrated
fig_cal.add_trace(go.Scatter(
    x=df_cal['time_rel'],
    y=df_cal['pt100_1_temp_calibrated'],
    mode='lines',
    name='PT100 #1 Calibrated',
    line=dict(color='#1f77b4', width=1),
))

# PT100 #2 calibrated
fig_cal.add_trace(go.Scatter(
    x=df_cal['time_rel'],
    y=df_cal['pt100_2_temp_calibrated'],
    mode='lines',
    name='PT100 #2 Calibrated',
    line=dict(color='#ff7f0e', width=1),
))

# Mark calibration points
fig_cal.add_trace(go.Scatter(
    x=[cal_ice_time_rel, cal_boil_time_rel],
    y=[cal_ice_temp, cal_boil_temp],
    mode='markers',
    name='Calibration Points',
    marker=dict(color='red', size=12, symbol='x'),
))

fig_cal.update_layout(
    title='Calibrated PT100 Temperatures (Two-Point Calibration)',
    xaxis_title='Time (seconds)',
    yaxis_title='Temperature (°C)',
    hovermode='x unified',
    legend=dict(yanchor="top", y=0.99, xanchor="left", x=0.01),
    dragmode='zoom',
)

fig_cal.show(config={'scrollZoom': True})

In [7]:
# Calculate and plot calibrated temperature delta
df_cal['temp_delta_calibrated'] = df_cal['pt100_1_temp_calibrated'] - df_cal['pt100_2_temp_calibrated']

print("=== Calibrated Temperature Delta Statistics ===")
print(f"Mean: {df_cal['temp_delta_calibrated'].mean():.4f} °C")
print(f"Std:  {df_cal['temp_delta_calibrated'].std():.4f} °C")
print(f"Min:  {df_cal['temp_delta_calibrated'].min():.4f} °C")
print(f"Max:  {df_cal['temp_delta_calibrated'].max():.4f} °C")

# Plot calibrated delta
fig_cal_delta = go.Figure()

fig_cal_delta.add_trace(go.Scatter(
    x=df_cal['time_rel'],
    y=df_cal['temp_delta_calibrated'],
    mode='lines',
    name='Delta (PT100 #1 - #2)',
    line=dict(color='#2ca02c', width=1),
))

fig_cal_delta.add_hline(y=0, line_dash="dash", line_color="gray", opacity=0.5)

fig_cal_delta.update_layout(
    title='Calibrated Temperature Delta (PT100 #1 - PT100 #2)',
    xaxis_title='Time (seconds)',
    yaxis_title='Temperature Delta (°C)',
    hovermode='x unified',
    dragmode='zoom',
)

fig_cal_delta.show(config={'scrollZoom': True})

=== Calibrated Temperature Delta Statistics ===
Mean: 0.9913 °C
Std:  3.2488 °C
Min:  -1.0206 °C
Max:  16.0593 °C
